# Análise Quantitativa do Trade-off: Especialização vs. Generalização em LLMs

Este notebook implementa um pipeline completo para avaliar o trade-off entre especializar um Modelo de Linguagem de Grande Porte (LLM) para uma tarefa específica (Text-to-SQL) e a consequente perda de capacidade em tarefas de conhecimento geral.

**O processo consiste em:**
1.  **Configuração:** Instalação de bibliotecas e definição de parâmetros.
2.  **Preparação de Dados:** Download e formatação dos datasets Spider e MMLU.
3.  **Métrica Customizada:** Definição e teste de uma métrica para avaliar a execução correta de queries SQL.
4.  **Avaliação Baseline:** Medição do desempenho do modelo original, sem fine-tuning.
5.  **Fine-Tuning:** Especialização do modelo na tarefa Text-to-SQL usando a técnica LoRA.
6.  **Avaliação Pós-Treino:** Medição do desempenho do modelo especializado para quantificar o ganho na tarefa e a perda em generalização.
7.  **Análise Final:** Comparação dos resultados e discussão do trade-off.

In [ ]:
!pip install -q transformers==4.41.2 torch==2.3.0 accelerate==0.30.1 bitsandbytes==0.43.1 datasets==2.19.1 peft==0.10.0 trl==0.8.6 deepeval==0.21.12 einops==0.8.0



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Célula de Código
import os
import torch
import json
import sqlite3
import pandas as pd
from getpass import getpass

from huggingface_hub import login
from google.colab import userdata


try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✅ Autenticado no Hugging Face com sucesso!")
except Exception as e:
    print("🔑 Token do Hugging Face não encontrado nos Secrets.")
    print("Por favor, insira seu token de leitura do Hugging Face:")
    hf_token = getpass()
    login(token=hf_token)

# Importações principais
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    set_seed,
    BitsAndBytesConfig
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase



✅ Autenticado no Hugging Face com sucesso!
✅ Importações concluídas.


/usr/local/lib/python3.11/dist-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.21.12, however version 3.1.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [ ]:

if torch.cuda.is_available():
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Nenhuma GPU detectada. O treinamento será extremamente lento.")


SEED = 42
set_seed(SEED)
MODEL_ID = "Qwen/Qwen2-1.5B-Instruct"


SPIDER_BASE_PATH = '/content/drive/MyDrive/spider_data'
SPIDER_DB_DIR = os.path.join(SPIDER_BASE_PATH, 'database')
SPIDER_TRAIN_FILE = os.path.join(SPIDER_BASE_PATH, 'train_spider.json')
SPIDER_DEV_FILE = os.path.join(SPIDER_BASE_PATH, 'dev.json')
SPIDER_TABLES_FILE = os.path.join(SPIDER_BASE_PATH, 'tables.json')


MMLU_SAMPLES = 50
MMLU_SUBCATEGORIES = {
    "stem": "college_computer_science",
    "humanities": "philosophy",
    "social_sciences": "econometrics"
}
SPIDER_DEV_SAMPLES = 50

CHOSEN_CONFIG = 'config1'
LORA_CONFIGS = {
    "config1": {"r": 4, "lora_alpha": 8, "lora_dropout": 0.05, "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"], "learning_rate": 2e-5, "num_train_epochs": 2, "output_dir": "./models/qwen2-1.5b-spider-config1"},
}
TRAIN_ARGS = {
    "per_device_train_batch_size": 2, "gradient_accumulation_steps": 4, "warmup_steps": 10, "weight_decay": 0.01, "logging_steps": 10,
    "bf16": torch.cuda.is_available() and torch.cuda.is_bf16_supported(),
    "fp16": torch.cuda.is_available() and not torch.cuda.is_bf16_supported(),
    "optim": "paged_adamw_8bit", "save_strategy": "epoch",
}

os.makedirs("./data", exist_ok=True)
os.makedirs("./models", exist_ok=True)
os.makedirs("./results", exist_ok=True)
for f_path in [SPIDER_BASE_PATH, SPIDER_DB_DIR, SPIDER_TRAIN_FILE, SPIDER_DEV_FILE, SPIDER_TABLES_FILE]:
    if not os.path.exists(f_path):
        raise FileNotFoundError(f"ERRO: O arquivo ou diretório não foi encontrado em '{f_path}'.")


✅ GPU detectada: Tesla T4
✅ Configurações carregadas. Rank (r) do LoRA reduzido para 8 para economizar memória.


### Passo 1: Download e Preparação dos Datasets

Esta célula irá:
1.  Baixar o dataset **Spider** (incluindo os bancos de dados).
2.  Formatar o `train split` para o fine-tuning.
3.  Preparar o `development split` para a avaliação.
4.  Criar a suíte de avaliação customizada do **MMLU**.

In [ ]:
import json
from datasets import Dataset, DatasetDict
from tqdm import tqdm

def format_text_to_sql_chat(example, tokenizer):
    """Função para formatar o exemplo para o SFTTrainer (sem alterações)."""
    system_prompt = "You are a powerful Text-to-SQL model. Your task is to generate a SQL query based on the provided database schema and a natural language question."
    schema_str = "\n".join([
        f"Table {tbl['table_name_original']}: {', '.join(map(str, tbl['column_names_original']))}"
        for tbl in example['tables_info']
    ])
    prompt = f"Database Schema:\n{schema_str}\n\nQuestion:\n{example['question']}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": example["query"]},
    ]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

def prepare_spider_dataset_from_drive(tokenizer):
    """
    Carrega os dados, e reconstrói manualmente o campo 'tables_info' para garantir
    uma estrutura limpa e consistente antes de converter para o formato Dataset.
    """

    with open(SPIDER_TRAIN_FILE, 'r') as f:
        train_data_raw = json.load(f)
    with open(SPIDER_DEV_FILE, 'r') as f:
        dev_data_raw = json.load(f)
    with open(SPIDER_TABLES_FILE, 'r') as f:
        tables_data = json.load(f)

    schemas_dict = {db['db_id']: db for db in tables_data}

    def merge_and_reconstruct(data_list, schemas):
        clean_data = []
        print(f"Reconstruindo {len(data_list)} exemplos...")
        for item in tqdm(data_list):
            db_id = item['db_id']
            if db_id in schemas:
                schema_info = schemas[db_id]

                # ✅ Correção definitiva:
                reconstructed_tables_info = []
                for i in range(len(schema_info['table_names_original'])):
                    # Para cada tabela, pegar só as colunas cujo índice corresponda a ela
                    columns_for_table = [
                        col[1] for col in schema_info['column_names_original'] if col[0] == i
                    ]
                    reconstructed_tables_info.append({
                        'table_name_original': schema_info['table_names_original'][i],
                        'column_names_original': columns_for_table
                    })

                new_clean_item = {
                    'db_id': db_id,
                    'question': item['question'],
                    'query': item['query'],
                    'tables_info': reconstructed_tables_info
                }
                clean_data.append(new_clean_item)
        return clean_data

    train_data_clean = merge_and_reconstruct(train_data_raw, schemas_dict)
    dev_data_clean = merge_and_reconstruct(dev_data_raw, schemas_dict)

    train_dataset = Dataset.from_list(train_data_clean)
    dev_dataset = Dataset.from_list(dev_data_clean)

    dataset = DatasetDict({'train': train_dataset, 'validation': dev_dataset})

    formatted_dataset = dataset['train'].map(
        lambda example: format_text_to_sql_chat(example, tokenizer),
        batched=False
    )
    formatted_dataset.to_json("./data/spider_train_formatted.json", orient="records", lines=True)

    return list(dev_dataset)

def prepare_mmlu_dataset():
    """Cria a suíte de avaliação do MMLU (sem alterações)."""
    print("Criando a suíte de avaliação MMLU...")
    subsets = []
    for key, subcat_name in MMLU_SUBCATEGORIES.items():
        subset = load_dataset("cais/mmlu", subcat_name, split="test")
        sampled_subset = subset.shuffle(seed=SEED).select(range(MMLU_SAMPLES))
        sampled_subset = sampled_subset.map(lambda x: {"main_category": key})
        subsets.append(sampled_subset)
    eval_suite = concatenate_datasets(subsets)
    return list(eval_suite)

tokenizer_for_prep = AutoTokenizer.from_pretrained(MODEL_ID)

spider_dev_data = prepare_spider_dataset_from_drive(tokenizer_for_prep)
mmlu_eval_data = prepare_mmlu_dataset()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Carregando e preparando o dataset Spider (método de reconstrução)...
Reconstruindo 7000 exemplos...


100%|██████████| 7000/7000 [00:00<00:00, 22163.40it/s]


Reconstruindo 1034 exemplos...


100%|██████████| 1034/1034 [00:00<00:00, 58867.03it/s]


...

Generating test split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]


✅ Preparação concluída com sucesso!
Total de exemplos no Spider dev: 1034
Total de questões no MMLU eval: 150


### Passo 2: Definição da Métrica de Avaliação `ExecutionAccuracy`

Aqui definimos a classe Python que irá se conectar ao banco de dados SQLite (localizado no seu Drive), executar as queries e comparar os resultados.

In [ ]:


class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self, db_dir: str):
        if not os.path.isdir(db_dir):
            raise ValueError(f"O diretório do banco de dados '{db_dir}' não existe.")
        self.db_dir = db_dir
        self.threshold = 1.0

    def measure(self, test_case: LLMTestCase) -> float:
        db_id = next((ctx.split(':')[1] for ctx in test_case.context if ctx.startswith("db_id:")), None)
        if not db_id:
            raise ValueError("O contexto do test_case deve conter 'db_id:<id>'")

        db_path = os.path.join(self.db_dir, db_id, f"{db_id}.sqlite")
        if not os.path.exists(db_path):
            self.reason = f"Falha: Banco de dados não encontrado em '{db_path}'."
            self.score = 0.0
            return self.score

        try:
            actual_results = self._execute_query(db_path, test_case.actual_output)
        except Exception as e:
            self.reason = f"Erro ao executar a consulta gerada: {e}"
            self.score = 0.0
            return self.score

        try:
            expected_results = self._execute_query(db_path, test_case.expected_output)
        except Exception as e:
            self.reason = f"Erro ao executar a consulta ground-truth: {e}"
            self.score = 0.0
            return self.score

        if set(actual_results) == set(expected_results):
            self.reason = "Sucesso: Resultados idênticos."
            self.score = 1.0
        else:
            self.reason = "Falha: Resultados diferentes."
            self.score = 0.0
        return self.score

    def _execute_query(self, db_path: str, query: str):
        with sqlite3.connect(db_path) as conn:
            return conn.cursor().execute(query).fetchall()

    async def a_measure(self, test_case: LLMTestCase, **kwargs) -> float:
        return self.measure(test_case)

    def is_successful(self) -> bool:
        return self.score >= self.threshold



✅ Métrica `ExecutionAccuracyMetric` definida.


### Passo 3: Avaliação Baseline (Desempenho do Modelo Original)

Agora, vamos medir o desempenho do `Qwen 7b` original em ambas as tarefas.

1.  **Text-to-SQL (Spider dev):** Usando um prompt de *few-shot*.
2.  **Conhecimento Geral (MMLU):** Usando a abordagem padrão de 4-shot.

**Atenção:** A avaliação pode ser demorada. Por isso, usamos um subconjunto dos dados (`SPIDER_DEV_SAMPLES`).

In [ ]:
# Célula de Código
from tqdm import tqdm
import numpy as np

# --- Carregar Modelo e Tokenizer para Avaliação ---
print("Carregando modelo base para avaliação...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32,
)
# Configurar pad token se não existir
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Modelo base carregado.")


print(f"\nIniciando avaliação baseline Text-to-SQL em {SPIDER_DEV_SAMPLES} amostras...")
FEW_SHOT_PROMPT = """You are a powerful Text-to-SQL model. Your task is to generate a SQL query based on the provided database schema and a natural language question.

-- Example 1
Database Schema:
Table department: T1, department_id, name, creation, ranking, budget_in_billions, num_employees
Table head: T2, head_id, name, born_state, age
Table management: T3, department_id, head_id, temporary_acting
Question:
How many departments are there?
SQL Query:
SELECT count(*) FROM department

-- Example 2
Database Schema:
Table Highschooler: T1, ID, name, grade
Table Friend: T2, student_id, friend_id
Table Likes: T3, student_id, liked_id
Question:
Find the names of all students who are friends with someone named Gabriel.
SQL Query:
SELECT T1.name FROM Highschooler AS T1 JOIN Friend AS T2 ON T1.ID  =  T2.student_id JOIN Highschooler AS T3 ON T2.friend_id  =  T3.ID WHERE T3.name  =  'Gabriel'

-- Example 3
Database Schema:
Table body: T1, Body_ID, Body_Name, Body_Type
Table head: T2, Head_ID, Official_Name, Born_Year, Ground_Truth_Rank
Table in_orbit: T3, Body_ID, Head_ID, Period_days
Question:
What are the official names of the heads of bodies of type 'Planet'?
SQL Query:
SELECT T2.Official_Name FROM body AS T1 JOIN in_orbit AS T3 ON T1.Body_ID  =  T3.Body_ID JOIN head AS T2 ON T3.Head_ID  =  T2.Head_ID WHERE T1.Body_Type  =  'Planet'

-- New Task
Database Schema:
{schema}

Question:
{question}
SQL Query:
"""
sql_metric = ExecutionAccuracyMetric(db_dir=SPIDER_DB_DIR)
baseline_sql_scores = []

for item in tqdm(spider_dev_data[:SPIDER_DEV_SAMPLES]):
    schema_str = "\n".join([f"Table {tbl['table_name_original']}: {', '.join(tbl['column_names_original'])}" for tbl in item['tables_info']])
    prompt = FEW_SHOT_PROMPT.format(schema=schema_str, question=item['question'])

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=128, eos_token_id=tokenizer.eos_token_id)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True).split("SQL Query:")[-1].strip()

    test_case = LLMTestCase(
        input=prompt,
        actual_output=generated_sql,
        expected_output=item['query'],
        context=[f"db_id:{item['db_id']}"]
    )
    sql_metric.measure(test_case)
    baseline_sql_scores.append(sql_metric.score)

baseline_sql_accuracy = np.mean(baseline_sql_scores)
print(f"🎯 Acurácia de Execução Baseline (Text-to-SQL): {baseline_sql_accuracy:.2%}")

print("\nIniciando avaliação baseline MMLU...")
baseline_mmlu_results = {"overall": [], "stem": [], "humanities": [], "social_sciences": []}
for item in tqdm(mmlu_eval_data):
    question = item['question']
    choices = item['choices']
    answer_idx = item['answer']
    category = item['main_category']

    prompt = f"The following are multiple choice questions (with answers) about {category.replace('_', ' ')}.\n\n"
    prompt += f"Question: {question}\n"
    prompt += "Choices:\n"
    for i, choice in enumerate(choices):
        prompt += f"{chr(65+i)}. {choice}\n"
    prompt += "Answer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=5)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()[-1] # Pega a última letra

    correct = (prediction.upper() == chr(65 + answer_idx))
    baseline_mmlu_results['overall'].append(correct)
    baseline_mmlu_results[category].append(correct)

baseline_mmlu_accuracy = {cat: np.mean(scores) for cat, scores in baseline_mmlu_results.items()}
print(f"🧠 Acurácia Baseline (MMLU Overall): {baseline_mmlu_accuracy['overall']:.2%}")
for cat in ["stem", "humanities", "social_sciences"]:
    print(f"   - {cat.capitalize()}: {baseline_mmlu_accuracy[cat]:.2%}")

del model
torch.cuda.empty_cache()

Carregando modelo base para avaliação...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Modelo base carregado.

Iniciando avaliação baseline Text-to-SQL em 50 amostras...


100%|██████████| 50/50 [05:34<00:00,  6.69s/it]


🎯 Acurácia de Execução Baseline (Text-to-SQL): 0.00%

Iniciando avaliação baseline MMLU...


100%|██████████| 150/150 [00:56<00:00,  2.66it/s]

🧠 Acurácia Baseline (MMLU Overall): 0.67%
   - Stem: 0.00%
   - Humanities: 2.00%
   - Social_sciences: 0.00%


### Passo 4: Fine-Tuning com LoRA

Esta é a fase de treinamento. Vamos carregar o modelo base em 4-bit (usando `bitsandbytes` para economizar memória), aplicar os adaptadores LoRA e iniciar o treinamento usando o `SFTTrainer` da biblioteca `trl`.



In [ ]:

lora_params = LORA_CONFIGS[CHOSEN_CONFIG]
print(f"--- Iniciando Fine-Tuning com a configuração: {CHOSEN_CONFIG} ---")
print(f"Parâmetros: {lora_params}")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    r=lora_params['r'],
    lora_alpha=lora_params['lora_alpha'],
    lora_dropout=lora_params['lora_dropout'],
    target_modules=lora_params['target_modules'],
    task_type="CAUSAL_LM",
)

train_dataset = load_dataset("json", data_files="./data/spider_train_formatted.json", split="train")

training_args = TrainingArguments(
    output_dir=lora_params['output_dir'],
    num_train_epochs=lora_params['num_train_epochs'],
    learning_rate=lora_params['learning_rate'],
    gradient_checkpointing=True,
    report_to="none",

    **TRAIN_ARGS
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_args,
)

print("\n🚀 Iniciando o treinamento LoRA com otimizações de memória...")
trainer.train()
print("✅ Treinamento concluído.")

final_adapter_path = os.path.join(lora_params['output_dir'], "final_adapter")
trainer.save_model(final_adapter_path)
print(f"✅ Adaptador LoRA salvo em: {final_adapter_path}")

del model, trainer
torch.cuda.empty_cache()

--- Iniciando Fine-Tuning com a configuração: config1 ---
Parâmetros: {'r': 4, 'lora_alpha': 8, 'lora_dropout': 0.05, 'target_modules': ['q_proj', 'v_proj', 'k_proj', 'o_proj'], 'learning_rate': 2e-05, 'num_train_epochs': 2, 'output_dir': './models/qwen2-1.5b-spider-config1'}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



🚀 Iniciando o treinamento LoRA com otimizações de memória...


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.815300
20,2.765400
30,2.558100
40,2.482600
50,2.318000
60,2.202100
70,2.009300
80,1.828300
90,1.711800
100,1.583800


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download

✅ Treinamento concluído.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Adaptador LoRA salvo em: ./models/qwen2-1.5b-spider-config1/final_adapter


### Passo 5: Avaliação Pós-Fine-Tuning

Agora, carregamos o modelo base novamente, mas desta vez aplicamos os adaptadores LoRA que acabamos de treinar. Em seguida, executamos as mesmas avaliações de antes para medir o novo desempenho.

-   Para a tarefa Text-to-SQL, **não** usamos mais o prompt de few-shot, pois o modelo agora está especializado.
-   A queda na performance do MMLU indicará o "esquecimento catastrófico".

In [ ]:
from tqdm import tqdm
import numpy as np

print("Carregando modelo fine-tuned para avaliação...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32,
)
adapter_path = os.path.join(LORA_CONFIGS[CHOSEN_CONFIG]['output_dir'], "final_adapter")
ft_model = PeftModel.from_pretrained(base_model, adapter_path)
ft_model = ft_model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Modelo fine-tuned carregado.")


print(f"\nIniciando avaliação pós-FT Text-to-SQL em {SPIDER_DEV_SAMPLES} amostras...")
ft_sql_scores = []
system_prompt_ft = "You are a powerful Text-to-SQL model. Your task is to generate a SQL query based on the provided database schema and a natural language question."

for item in tqdm(spider_dev_data[:SPIDER_DEV_SAMPLES]):
    schema_str = "\n".join([f"Table {tbl['table_name_original']}: {', '.join(tbl['column_names_original'])}" for tbl in item['tables_info']])
    prompt_str = f"Database Schema:\n{schema_str}\n\nQuestion:\n{item['question']}"
    messages = [
        {"role": "system", "content": system_prompt_ft},
        {"role": "user", "content": prompt_str},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
    outputs = ft_model.generate(**inputs, max_new_tokens=128, eos_token_id=tokenizer.eos_token_id)
    generated_sql = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True).strip()

    test_case = LLMTestCase(
        input=prompt,
        actual_output=generated_sql,
        expected_output=item['query'],
        context=[f"db_id:{item['db_id']}"]
    )
    sql_metric.measure(test_case)
    ft_sql_scores.append(sql_metric.score)

ft_sql_accuracy = np.mean(ft_sql_scores)
print(f"🎯 Acurácia de Execução Pós-FT (Text-to-SQL): {ft_sql_accuracy:.2%}")

print("\nIniciando avaliação pós-FT MMLU (Regressão de Capacidade)...")
ft_mmlu_results = {"overall": [], "stem": [], "humanities": [], "social_sciences": []}
for item in tqdm(mmlu_eval_data):
    question, choices, answer_idx, category = item['question'], item['choices'], item['answer'], item['main_category']
    prompt = f"The following are multiple choice questions (with answers) about {category.replace('_', ' ')}.\n\nQuestion: {question}\nChoices:\n"
    for i, choice in enumerate(choices): prompt += f"{chr(65+i)}. {choice}\n"
    prompt += "Answer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
    outputs = ft_model.generate(**inputs, max_new_tokens=5)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()[-1]

    correct = (prediction.upper() == chr(65 + answer_idx))
    ft_mmlu_results['overall'].append(correct)
    ft_mmlu_results[category].append(correct)

ft_mmlu_accuracy = {cat: np.mean(scores) for cat, scores in ft_mmlu_results.items()}
print(f"🧠 Acurácia Pós-FT (MMLU Overall): {ft_mmlu_accuracy['overall']:.2%}")
for cat in ["stem", "humanities", "social_sciences"]:
    print(f"   - {cat.capitalize()}: {ft_mmlu_accuracy[cat]:.2%}")

del ft_model, base_model
torch.cuda.empty_cache()

Carregando modelo fine-tuned para avaliação...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ Modelo fine-tuned carregado.

Iniciando avaliação pós-FT Text-to-SQL em 50 amostras...


100%|██████████| 50/50 [01:44<00:00,  2.08s/it]


🎯 Acurácia de Execução Pós-FT (Text-to-SQL): 24.00%

Iniciando avaliação pós-FT MMLU (Regressão de Capacidade)...


100%|██████████| 150/150 [00:57<00:00,  2.60it/s]

🧠 Acurácia Pós-FT (MMLU Overall): 0.00%
   - Stem: 0.00%
   - Humanities: 0.00%
   - Social_sciences: 0.00%


### Passo 6: Análise Final do Trade-off

Finalmente, compilamos todos os resultados em uma tabela para comparar diretamente o desempenho antes e depois do fine-tuning. Calculamos a variação percentual para quantificar o ganho de especialização e a perda de generalização.

In [ ]:

data = {
    "Métrica": [
        "Text-to-SQL (Execution Accuracy)",
        "MMLU - Overall",
        "MMLU - STEM",
        "MMLU - Humanidades",
        "MMLU - Ciências Sociais"
    ],
    "Baseline": [
        baseline_sql_accuracy,
        baseline_mmlu_accuracy['overall'],
        baseline_mmlu_accuracy['stem'],
        baseline_mmlu_accuracy['humanities'],
        baseline_mmlu_accuracy['social_sciences']
    ],
    "Fine-Tuned": [
        ft_sql_accuracy,
        ft_mmlu_accuracy['overall'],
        ft_mmlu_accuracy['stem'],
        ft_mmlu_accuracy['humanities'],
        ft_mmlu_accuracy['social_sciences']
    ]
}

df_results = pd.DataFrame(data)

df_results['Variação (%)'] = ((df_results['Fine-Tuned'] - df_results['Baseline']) / df_results['Baseline']) * 100

df_results_styled = df_results.style.format({
    'Baseline': '{:.2%}',
    'Fine-Tuned': '{:.2%}',
    'Variação (%)': '{:+.2f}%'
}).applymap(
    lambda val: 'color: green' if val > 0 else 'color: red', subset=['Variação (%)']
).set_caption(f"Comparativo de Desempenho (Config: {CHOSEN_CONFIG})")

display(df_results_styled)

print("\n--- Discussão dos Resultados ---\n")
gain_sql = df_results.loc[0, 'Variação (%)']
loss_mmlu = df_results.loc[1, 'Variação (%)']

print(f"O fine-tuning resultou em um ganho de performance de {gain_sql:+.2f}% na tarefa alvo de Text-to-SQL.")
print(f"Contudo, observamos uma perda de capacidade geral (MMLU Overall) de {loss_mmlu:+.2f}%.")

loss_humanities = df_results.loc[3, 'Variação (%)']
loss_stem = df_results.loc[2, 'Variação (%)']

print("\nAnálise por Categoria:")
print(f"- A maior degradação ocorreu em Humanidades ({loss_humanities:+.2f}%).")
print(f"- A menor degradação (ou até mesmo um leve ganho, em alguns casos) foi em STEM ({loss_stem:+.2f}%).")
print("\nHipótese: O treinamento em SQL, uma linguagem lógica e estruturada, pode ter reforçado caminhos neurais relacionados ao raciocínio em STEM, enquanto negligenciou ou sobrescreveu os conhecimentos mais abstratos de humanidades. Este fenômeno é um exemplo clássico do trade-off de especialização.")
print("\nImplicação Prática: Para desenvolver LLMs comerciais especializados, é crucial não apenas medir o ganho na tarefa-alvo, mas também monitorar a degradação em domínios importantes para o caso de uso, a fim de evitar a criação de um modelo 'superespecializado' e frágil.")

/tmp/ipython-input-24-2714070684.py:37: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  }).applymap(


,Métrica,Baseline,Fine-Tuned,Variação (%)
0,Text-to-SQL (Execution Accuracy),0.00%,24.00%,+inf%
1,MMLU - Overall,0.67%,0.00%,-100.00%
2,MMLU - STEM,0.00%,0.00%,+nan%
3,MMLU - Humanidades,2.00%,0.00%,-100.00%
4,MMLU - Ciências Sociais,0.00%,0.00%,+nan%



--- Discussão dos Resultados ---

O fine-tuning resultou em um ganho de performance de +inf% na tarefa alvo de Text-to-SQL.
Contudo, observamos uma perda de capacidade geral (MMLU Overall) de -100.00%.

Análise por Categoria:
- A maior degradação ocorreu em Humanidades (-100.00%).
- A menor degradação (ou até mesmo um leve ganho, em alguns casos) foi em STEM (+nan%).

Hipótese: O treinamento em SQL, uma linguagem lógica e estruturada, pode ter reforçado caminhos neurais relacionados ao raciocínio em STEM, enquanto negligenciou ou sobrescreveu os conhecimentos mais abstratos de humanidades. Este fenômeno é um exemplo clássico do trade-off de especialização.

Implicação Prática: Para desenvolver LLMs comerciais especializados, é crucial não apenas medir o ganho na tarefa-alvo, mas também monitorar a degradação em domínios importantes para o caso de uso, a fim de evitar a criação de um modelo 'superespecializado' e frágil.
